In [1]:
import numpy as np
from fmpy import *
from pyfmi import load_fmu
import matplotlib.pyplot as plt
import random

In [2]:
fmu = 'ego.fmu'
dump(fmu)
model = load_fmu(fmu)


Model Info

  FMI Version        2.0
  FMI Type           Co-Simulation
  Model Name         ego
  Description        None
  Platforms          linux64, win64
  Continuous States  0
  Event Indicators   0
  Variables          16
  Generation Tool    Simulink (R2024b)
  Generation Date    2025-01-28T17:34:10Z

Default Experiment

  Stop Time          5.0
  Step Size          0.1

Variables (input, output)

  Name               Causality              Start Value  Unit     Description
  X                  input                            0           X
  Y                  input                            0           Y
  Z                  input                            0           Z
  Heading            input                            0           Heading
  Pitch              input                            0           Pitch
  Roll               input                            0           Roll
  Speed              input                            0           Speed
  timestamps       

In [3]:
model_description = read_model_description(fmu)

# 提取输入变量
inputs = []
for variable in model_description.modelVariables:
    if variable.causality == 'input':  # 输入变量
        inputs.append(variable.name)

# 提取输出变量
outputs = []
for variable in model_description.modelVariables:
    if variable.causality == 'output':  # 输出变量
        outputs.append(variable.name)

print("输入变量:", inputs)
print("输出变量:", outputs)

输入变量: ['X', 'Y', 'Z', 'Heading', 'Pitch', 'Roll', 'Speed']
输出变量: ['timestamps', 'location[1,1]', 'location[1,2]', 'location[1,3]', 'angle[1,1]', 'angle[1,2]', 'angle[1,3]', 'accelerate_']


In [4]:
np.random.seed(42)
start_time = 0
stop_time = 5
step_size = 0.1

timestamps = np.arange(start_time, stop_time, step_size)

# 生成整数数据
X = np.random.randint(-10, 11, size=len(timestamps))
Y = np.random.randint(-10, 11, size=len(timestamps))
Z = np.random.randint(-10, 11, size=len(timestamps))
heading = np.random.randint(-90, 91, size=len(timestamps))
pitch = np.random.randint(-90, 91, size=len(timestamps))
roll = np.random.randint(-90, 91, size=len(timestamps))
speed = np.random.randint(50, 101, size=len(timestamps))

# 组合数据，确保第一列是时间戳
input_data = np.column_stack((timestamps, X, Y, Z, heading, pitch, roll, speed))

# 变量名称
input_names = ["X", "Y", "Z", "Heading", "Pitch", "Roll", "Speed"]

# 传入 pyfmi 的正确格式
input_tuple = (input_names, input_data)

In [6]:
result = model.simulate(start_time=start_time, final_time=stop_time,
                        options={'ncp': int((stop_time - start_time) / step_size)},
                        input=input_tuple)

Simulation interval    : 0 - 5.0 seconds.
Elapsed simulation time: 0.037256398994941264 seconds.


In [7]:
result_timestamp = result[result.dtype.names[1]]  # Assuming 'timestamp' is the correct field name
result_location = np.column_stack((result[result.dtype.names[2]], result[result.dtype.names[3]], result[result.dtype.names[4]]))  # Combine X and Y for location
result_angle = np.column_stack((result[result.dtype.names[5]], result[result.dtype.names[6]], result[result.dtype.names[7]]))  # Combine heading and pitch for angle
result_speed = result[result.dtype.names[-1]]  # Access speed directly

# print(result_location)

AttributeError: 'FMIResult' object has no attribute 'dtype'

In [8]:
print(result)